# WSI - ćwiczenie 1.
### Zagadnienie przeszukiwania i podstawowe podejścia do niego


In [70]:
import numpy as np
from plotly import graph_objs as go

1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [71]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

### Algorytm najszybszego spadku


In [72]:
def gradient_descent(start_point, beta, grad, num_iters):
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        point = point - beta * theta
        steps = np.append(steps, [point], 0)
    return steps


### generowanie wykresów

In [86]:
max_r = 2.1
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])
steps = gradient_descent(np.array([2,2]), 0.001, grad_g, 100)
p = np.array([pt for pt in steps if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
XS, YS = p[:,0], p[:,1]
ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

layout = go.Layout(width = 700, height =700,
                             title_text='Gradient Descent')
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                 opacity=0.5)], layout=layout)
fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='markers', 
                  marker=dict(size=4, color=ZS,               
                              colorscale='Aggrnyl'))
fig.show()

In [74]:
def main():
    pass
if __name__=="__main__":
    main()